
# Page.prototype.setData(Object data, Function callback)
setData 函数用于将数据从逻辑层发送到视图层（异步），同时改变对应的 this.data 的值（同步）。

| 字段     | 类型     | 必填 | 描述                                         | 最低版本 |
|----------|----------|------|----------------------------------------------|----------|
| data     | Object   | 是   | 这次要改变的数据                             |          |
| callback | Function | 否   | setData引起的界面更新渲染完毕后的回调函数    | 1.5.0    |

Object 以 key: value 的形式表示，将 this.data 中的 key 对应的值改变成 value。

**其中 key 可以以数据路径的形式给出，支持改变数组中的某一项或对象的某个属性，如 array[2].message，a.b.c.d，并且不需要在 this.data 中预先定义。**
- 注意：

- 直接修改 this.data 而不调用 this.setData 是无法改变页面的状态的，还会造成数据不一致。
- 仅支持设置可 JSON 化的数据。
- 单次设置的数据不能超过`1024kB`，请尽量避免一次设置过多的数据。
- 请不要把 data 中任何一项的 value 设为 undefined ，否则这一项将不被设置并可能遗留一些潜在问题。

## 1. setData 的流程
setData 的过程，大致可以分成几个阶段：

- 逻辑层虚拟 DOM 树的遍历和更新，触发组件生命周期和 observer 等；
- 将 data 从逻辑层传输到视图层；
- 视图层虚拟 DOM 树的更新、真实 DOM 元素的更新并触发页面渲染更新。

## 2. 数据通信

- 需要进行数据的序列化、跨线程/进程的数据传输、数据的反序列化，因此数据传输过程是异步的、非实时的。
- 数据传输的耗时与数据量的大小正相关，如果对端线程处于繁忙状态，数据会在消息队列中等待。



## 3. 使用建议

### 3.1 data 应只包括渲染相关的数据
setData 应只用来进行渲染相关的数据更新。用 setData 的方式更新渲染无关的字段，会触发额外的渲染流程，或者增加传输的数据量，影响渲染耗时。

- ✅ 页面或组件的 data 字段，应用来存放和页面或组件渲染相关的数据（即直接在 wxml 中出现的字段）；
- ✅ 页面或组件渲染间接相关的数据可以设置为「纯数据字段」，可以使用 setData 设置并使用 observers 监听变化；
- ✅ 页面或组件渲染无关的数据，应挂在非 data 的字段下，如 this.userData = {userId: 'xxx'}；
- ❌ 避免在 data 中包含渲染无关的业务数据；
- ❌ 避免使用 data 在页面或组件方法间进行数据共享；
- ❌ 避免滥用 纯数据字段 来保存可以使用非 data 字段保存的数据。


### 3.2 控制 setData 的频率

每次 setData 都会触发逻辑层虚拟 DOM 树的遍历和更新，也可能会导致触发一次完整的页面渲染流程。过于频繁（毫秒级）的调用 setData，会导致以下后果：

- 逻辑层 JS 线程持续繁忙，无法正常响应用户操作的事件，也无法正常完成页面切换；
- 视图层 JS 线程持续处于忙碌状态，逻辑层 -> 视图层通信耗时上升，视图层收到消息的延时较高，渲染出现明显延迟；
- 视图层无法及时响应用户操作，用户滑动页面时感到明显卡顿，操作反馈延迟，用户操作事件无法及时传递到逻辑层，逻辑层亦无法及时将操作处理结果及时传递到视图层。

因此，开发者在调用 setData 时要注意：

- ✅ 仅在需要进行页面内容更新时调用 setData；
- ✅ 对连续的 setData 调用尽可能的进行合并；
- ❌ 避免不必要的 setData；
- ❌ 避免以过高的频率持续调用 setData，例如毫秒级的倒计时；
- ❌ 避免在 onPageScroll 回调中每次都调用 setData。


### 3.3 选择合适的 setData 范围

组件的 setData 只会引起当前组件和子组件的更新，可以降低虚拟 DOM 更新时的计算开销。
- ✅ 对于需要频繁更新的页面元素（例如：秒杀倒计时），可以封装为独立的组件，在组件内进行 setData 操作。必要时可以使用 CSS contain 属性限制计算布局、样式和绘制等的范围。

### 3.4 setData 应只传发生变化的数据

setData 的数据量会影响数据拷贝和数据通讯的耗时，增加页面更新的开销，造成页面更新延迟。

- ✅ setData 应只传入发生变化的字段；
- ✅ 建议以数据路径形式改变数组中的某一项或对象的某个属性，如 this.setData({'array[2].message': 'newVal', 'a.b.c.d': 'newVal'})，而不是每次都更新整个对象或数组；
- ❌ 不要在 setData 中偷懒一次性传所有data：this.setData(this.data)。



### 4. 性能分析
开发者可以通过组件的 setUpdatePerformanceListener 接口获取更新性能统计信息，来分析产生性能瓶颈的组件。






```js
<view>
  <text>{{userName}}</text>
  <button bindtap="loadUserInfo">加载用户信息</button>
</view>
Page({
  // 1. 渲染相关数据，出现在 WXML
  data: {
    userName: '未登录',             // 直接用于渲染
    'pure.userLoaded': false       // 纯数据字段，用于观察变化
  },

  // 2. 非 data 字段：完全与渲染无关，仅业务逻辑用
  userData: {
    userId: '123',
    userInfo: null
  },

  // 3. 配置纯数据字段：匹配以 "pure." 开头的字段不参与渲染
  options: {
    pureDataPattern: /^pure\./
  },

  // 4. 监听纯数据字段
  observers: {
    'pure.userLoaded'(val) {
      if (val) {
        console.log('用户加载完成，执行逻辑');
      }
    }
  },

  // 5. 事件处理
  loadUserInfo() {
    // 假设从服务端加载用户信息
    setTimeout(() => {
      this.userData.userInfo = {
        age: 18,
        address: 'Beijing'
      }

      // 设置纯数据字段，触发 observers 但不影响渲染
      this.setData({
        'pure.userLoaded': true
      });
    }, 1000);
  }
});
```